In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
import os
import IPython.display as display
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile

# Load and process data

In [2]:
def columns_to_y(row):
  labels = ["Marker1", "Marker2", "Marker3", "Marker4",
            "ColorLeft", "ColorRight", "GrayScaleRight", "GreyScaleLeft",
            "CircleTop", "CircleRight", "CircleBottom", "CircleLeft"]

  output = []
  for label in labels:
      for coord in eval(row[label]):
        output.append(float(coord))
  return output

def name_to_path(row, prefix):
  path = prefix + row["photoFullPath"].split("/")[-1].lower()
  return path

def load_image(row):
  img = matplotlib.image.imread(row["path"])
  # if len(img.shape) != 3:
  #  img = np.stack((img,)*3, axis=-1)
  return tf.convert_to_tensor(img, dtype=tf.float32)

In [3]:
frames = []

with zipfile.ZipFile("./drive/My Drive/Colab Notebooks/dataset.zip", 'r') as zip_ref:
  zip_ref.extractall(".")
print(len(os.listdir("dataset")))
for i in range(len(os.listdir("dataset"))):
  data = pd.read_csv("dataset/data{}/labeled_data.csv".format(i), sep="|")
  data["path"] = data.apply(lambda row: name_to_path(row, "./dataset/data{}/source_to_label/".format(i)), axis=1)
  data["y"] = data.apply(lambda row: columns_to_y(row), axis=1)
  frames.append(data[["path", "y"]])

# with zipfile.ZipFile("./drive/My Drive/Colab Notebooks/validation.zip", 'r') as zip_ref:
#     zip_ref.extractall(".")
# validation_data = pd.read_csv("validation/data/data.txt", sep="|", usecols=["photoFullPath1", "LeftTop", "RightTop", "RightLow", "LeftLow"])
# validation_data["path"] = validation_data.apply(lambda row: name_to_path(row, "validation/"), axis=1)
# validation_data["y"] = validation_data.apply(lambda row: columns_to_y(row), axis=1)
# data, _ = train_test_split(validation_data, test_size=0.5)
# frames.append(data[["path", "y"]])

13


In [4]:
data = pd.concat(frames, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)
#validation_data = validation_data.sample(frac=1).reset_index(drop=True)
data, validation_data = train_test_split(data, test_size=0.1)
data = data.sample(frac=1).reset_index(drop=True)
validation_data = validation_data.sample(frac=1).reset_index(drop=True)
print(len(data))
print(len(validation_data))

3060
340


In [5]:
print(data)
print(validation_data["y"].iloc[0])

                                          path                                                  y
0      ./dataset/data7/source_to_label/397.jpg  [0.23, 0.7266666666666667, 0.2175, 0.298333333...
1      ./dataset/data3/source_to_label/121.jpg  [0.235, 0.24833333333333332, 0.73, 0.243333333...
2      ./dataset/data6/source_to_label/796.jpg  [0.1675, 0.8666666666666667, 0.1525, 0.1066666...
3      ./dataset/data7/source_to_label/257.jpg  [0.235, 0.655, 0.2425, 0.15833333333333333, 0....
4      ./dataset/data7/source_to_label/353.jpg  [0.28, 0.6933333333333334, 0.245, 0.2533333333...
...                                        ...                                                ...
3055   ./dataset/data6/source_to_label/756.jpg  [0.1675, 0.6683333333333333, 0.2125, 0.105, 0....
3056  ./dataset/data11/source_to_label/113.jpg  [0.13, 0.7133333333333334, 0.1975, 0.108333333...
3057   ./dataset/data2/source_to_label/143.jpg  [0.2175, 0.195, 0.7275, 0.19, 0.7475, 0.753333...
3058   ./dataset/dat

In [6]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch_indices = [self.indices[k] for k in index]
        
        X, Y = self.__get_data(batch_indices)
        return X, Y

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle:
            np.random.shuffle(self.index)

    def __get_data(self, batch_indices):
        batch = self.df.iloc[batch_indices,:]
        batch["image"] = batch.apply(lambda row: load_image(row), axis=1)
        X = tf.convert_to_tensor(batch["image"].tolist())
        Y = tf.convert_to_tensor(batch["y"].tolist())
        return X, Y


In [7]:
dataset = DataGenerator(data)
val_dataset = DataGenerator(validation_data)

# Create Model

**Create model**

In [8]:
mobile_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

inputs = tf.keras.Input(shape=(224, 224, 3))
x = mobile_model(inputs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(.1)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(.1)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(.1)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(.2)(x)
outputs = tf.keras.layers.Dense(24, activation='sigmoid')(x)
detector_model = tf.keras.Model(inputs, outputs)
detector_model.compile(optimizer="Adam", loss="mean_squared_error")

detector_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten (Flatten)            (None, 62720)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              64226304  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout (Dropout)            (None, 1024)             

**Load model**

In [ ]:
detector_model = tf.keras.models.load_model("./drive/My Drive/Colab Notebooks/mobile1.hdf5")

In [9]:
detector_model.evaluate(val_dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


10/10 [==============================] - 1s 65ms/step - loss: 0.0618


0.06184384226799011

In [ ]:
detector_model.fit(dataset, validation_data=val_dataset, epochs=200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/200
95/95 [==============================] - 22s 228ms/step - loss: 0.0311 - val_loss: 0.1281
Epoch 2/200
95/95 [==============================] - 22s 231ms/step - loss: 0.0090 - val_loss: 0.0523
Epoch 3/200
95/95 [==============================] - 22s 233ms/step - loss: 0.0068 - val_loss: 0.0796
Epoch 4/200
95/95 [==============================] - 22s 236ms/step - loss: 0.0061 - val_loss: 0.0139
Epoch 5/200
95/95 [==============================] - 23s 240ms/step - loss: 0.0061 - val_loss: 0.0340
Epoch 6/200
95/95 [==============================] - 23s 244ms/step - loss: 0.0059 - val_loss: 0.0308
Epoch 7/200
95/95 [==============================] - 23s 243ms/step - loss: 0.0053 - val_loss: 0.0658
Epoch 8/200
95/95 [==============================] - 23s 241ms/step - loss: 0.0049 - val_loss: 0.0087
Epoch 9/200
95/95 [==============================] - 23s 241ms/step - loss: 0.0046 - val_loss: 0.0120
Epoch 10/200
95/95 [==============================] - 23s 242ms/step - loss: 0.004

In [ ]:
detector_model.save("./drive/My Drive/Colab Notebooks/mobile1.hdf5")  

# Tego niżej nie używamy

In [ ]:
opt = tf.optimizers.Adam()

@tf.function
def train_step(image, y):
  with tf.GradientTape() as tape:
    output = detector_model(image)
    loss = tf.math.reduce_mean(tf.math.squared_difference(y, output))
  grad = tape.gradient(loss, detector_model.trainable_weights)
  opt.apply_gradients(zip(grad, detector_model.trainable_weights))
  return loss

epochs = 40
steps_per_epoch = len(dataset)
display.display("loss = {}".format(0), display_id=44484848484)
for n in range(epochs):
  step = 0
  total_loss = 0
  for m in range(steps_per_epoch):
    step += 1 
    image, y = dataset[m]
    loss = train_step(image, y)
    total_loss += loss
    print(".", end='')
    display.update_display("loss = {}".format(total_loss/step), display_id=44484848484)
  
  detector_model.save("./drive/My Drive/Colab Notebooks/model1.hdf5")
  display.clear_output(wait=True)
  display.display("epoch {}/{}".format(n, epochs))
